In [56]:
import keras
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import os, json

I drop time related features because I do not care about it when creating my embeddings

In [24]:
df = pd.read_csv('MyData/biggest_playlist_ever.csv')
df.drop(columns = ['Unnamed: 0','duration_ms','time_signature'],inplace=True)

In [25]:
df.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo
0,Hozier,Hozier (Deluxe),Take Me To Church,7dS5EaCoMnN7DzlpT6aRn2,0.566,0.664,4,-5.303,0,0.0464,0.000000,0.116,0.437,128.945
1,Mike Posner,31 Minutes to Takeoff,Cooler Than Me - Single Mix,2V4bv1fNWfTcyRJKmej6Sj,0.768,0.820,7,-4.630,0,0.0475,0.000000,0.704,0.625,129.969
2,"Tyler, The Creator",Flower Boy,See You Again (feat. Kali Uchis),7KA4W4McWYRpgf0fWsJZWB,0.558,0.559,6,-9.222,1,0.0959,0.000007,0.109,0.620,78.558
3,Bastille,Bad Blood,Pompeii,3gbBpTdY8lnQwqxNCcf795,0.679,0.715,9,-6.383,1,0.0407,0.000000,0.271,0.571,127.435
4,Shakira,"Oral Fixation, Vol. 2 (Expanded Edition)",Hips Don't Lie (feat. Wyclef Jean),3ZFTkvIE7kyPt6Nu3PEa7V,0.778,0.824,10,-5.892,0,0.0712,0.000000,0.405,0.756,100.024


Get artists that have at least 3 songs in the dataframe as we should have at least 3 data points to create the embeddings

In [50]:
artists = df['artist'].value_counts()
artists = artists[df['artist'].value_counts()>2]
artists = artists.index

In [52]:
df = df[df['artist'].isin(artists)]

In [55]:
df.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo
1,Mike Posner,31 Minutes to Takeoff,Cooler Than Me - Single Mix,2V4bv1fNWfTcyRJKmej6Sj,0.768,0.820,7,-4.630,0,0.0475,0.0000,0.704,0.625,129.969
4,Shakira,"Oral Fixation, Vol. 2 (Expanded Edition)",Hips Don't Lie (feat. Wyclef Jean),3ZFTkvIE7kyPt6Nu3PEa7V,0.778,0.824,10,-5.892,0,0.0712,0.0000,0.405,0.756,100.024
5,The Script,#3 Deluxe Version,Hall of Fame (feat. will.i.am),7wMq5n8mYSKlQIGECKUgTX,0.421,0.873,10,-4.343,1,0.0565,0.0000,0.123,0.630,84.803
6,Otto Knows,Back Where I Belong (feat. Avicii),Back Where I Belong (feat. Avicii),78W8wiUIlQ2SnWY9TVowKZ,0.497,0.865,0,-3.190,1,0.0644,0.0679,0.443,0.595,125.960
7,Selena Gomez,The Heart Wants What It Wants,The Heart Wants What It Wants,2jcPoPin36hyNRSi4ZsqeB,0.616,0.789,7,-4.874,0,0.0377,0.0000,0.142,0.621,83.066


Label encode the artist names so we can pass it to our embedding model in the future

In [57]:
le = LabelEncoder()

In [58]:
x_train = le.fit_transform(df['artist'])

In [65]:
x_train

array([270, 349, 374, ..., 115, 188, 188])

In [69]:
y_train = df.iloc[:,4:]

In [70]:
y_train.head()

,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo
1,0.768,0.820,7,-4.630,0,0.0475,0.0000,0.704,0.625,129.969
4,0.778,0.824,10,-5.892,0,0.0712,0.0000,0.405,0.756,100.024
5,0.421,0.873,10,-4.343,1,0.0565,0.0000,0.123,0.630,84.803
6,0.497,0.865,0,-3.190,1,0.0644,0.0679,0.443,0.595,125.960
7,0.616,0.789,7,-4.874,0,0.0377,0.0000,0.142,0.621,83.066


### Now its time for the fun stuff. Let's define a function to build our embedding model